In [3]:
def image_traintest():
    import os
# 이미지 파일 이름 추출
    train_img_files = set([filename.split(".")[0] for filename in os.listdir(target_directory)])
    test_img_files = set([filename.split(".")[0] for filename in os.listdir(target_directory_test)])

# 공통된 이름 찾기
    common_names = train_img_files.intersection(test_img_files)

# 이미지 총합 계산
    train_img_total = len(train_img_files)
    test_img_total = len(test_img_files)
    img_total = train_img_total + test_img_total

    if common_names:
        print("같은 이름을 가진 이미지들:")
        for name in common_names:
            print(name)
    else:
        print("같은 이름을 가진 이미지가 없습니다.")

    train_ratio = (train_img_total / img_total) * 100
    test_ratio = (test_img_total / img_total) * 100

    print(f"train_img_total: {train_img_total} ({train_ratio:.2f}%)")
    print(f"test_img_total: {test_img_total} ({test_ratio:.2f}%)")
    print(f"img_total: {img_total}")

def trade_test_model(): 

    from collections import defaultdict
    import random
    import os
    import shutil
    import os
    import xml.etree.ElementTree as ET

    def preprocess_data(data_dit, ratio):
    # 데이터를 딕셔너리로 변환
        filename_object_dict = []
        for filename, object_names in zip(data_dit["Filename"], data_dit["Object Names"]):
            object_names = object_names.split(", ")
            filename_object_dict.append({"filename": filename, "object_names": object_names})

    # 데이터를 랜덤하게 섞음
        random.shuffle(filename_object_dict)

    # 데이터를 지정된 비율로 나눔
        split_index = int(len(filename_object_dict) * ratio)
        train_data = filename_object_dict[:split_index]
        test_data = filename_object_dict[split_index:]

    # 중복 없는 객체 이름 목록 생성
        object_name_list = set()

        for entry in filename_object_dict:
            object_names = entry['object_names']
            object_name_list.update(object_names)

        object_name_list = list(object_name_list)  # 세트를 리스트로 변환

    # train_data를 딕셔너리 형식으로 변환
        train_data_dict = {}
        for entry in train_data:
            train_data_dict[entry['filename']] = entry['object_names']

    # test_data를 딕셔너리 형식으로 변환
        test_data_dict = {}
        for entry in test_data:
            test_data_dict[entry['filename']] = entry['object_names']

        return train_data_dict, test_data_dict, object_name_list

    def calculate_percentage(data_dit, object_name_list):
        results = {}

        for object_name in object_name_list:
            variable_name = object_name.replace(" ", "_")
            variable_total_name = f"{variable_name}_total"
            variable_totalt_name = f"{variable_name}_totalt"
        
            if variable_total_name in globals() and variable_totalt_name in globals():
                object_total = globals()[variable_total_name]
                object_totalt = globals()[variable_totalt_name]
                percentage = (object_total / (object_total + object_totalt)) * 100
                results[object_name] = percentage
    
        max_percentage = max(results.values())
        max_percentage_objects = [name for name, percentage in results.items() if percentage == max_percentage]
    
        return results, max_percentage_objects

    def print_results_with_threshold(results, max_percentage_objects, threshold):
        filtered_results = {name: percentage for name, percentage in results.items() if percentage > threshold}

        if filtered_results:
            max_percentage = max(filtered_results.values())
            max_percentage_objects_str = ', '.join(max_percentage_objects)

        
            print(f"\nObject with the highest percentage above {threshold}%: {max_percentage_objects_str}  {max_percentage:.2f}%")
        else:
            print(f"No objects with percentage above {threshold}% found.")
        
    def calculate_percentage_test(data_dit, object_name_list):
        results_test = {}

        for object_name in object_name_list:
            variable_name = object_name.replace(" ", "_")
            variable_total_name = f"{variable_name}_total"
            variable_totalt_name = f"{variable_name}_totalt"
        
            if variable_total_name in globals() and variable_totalt_name in globals():
                object_total = globals()[variable_total_name]
                object_totalt = globals()[variable_totalt_name]
                percentage = (object_totalt / (object_total + object_totalt)) * 100
                results_test[object_name] = percentage
    
        max_percentage_test = max(results_test.values())
        max_percentage_objects_test = [name for name, percentage in results_test.items() if percentage == max_percentage_test]
    
        return results_test, max_percentage_objects_test

    def print_results_with_threshold_test(results_test, max_percentage_objects_test, threshold):
        filtered_results_test = {name: percentage for name, percentage in results_test.items() if percentage > threshold}

        if filtered_results:
            max_percentage_test = max(filtered_results_test.values())
            max_percentage_objects_str_test = ', '.join(max_percentage_objects_test)

        
            print(f"\nObject with the highest percentage above {threshold}%: {max_percentage_objects_str_test}  {max_percentage_test:.2f}%")
        else:
            print(f"No objects with percentage above {threshold}% found.")


    def read_xml_files(xml_folder_path):
        xml_files = []

        for root, _, files in os.walk(xml_folder_path):
            for filename in files:
                if filename.endswith(".xml"):
                    xml_path = os.path.join(root, filename)
                    xml_files.append(xml_path)

        return xml_files

    def parse_objects_in_file(xml_path):
        tree = ET.parse(xml_path)
        root_element = tree.getroot()

        objects = root_element.findall("object")
        object_names = [obj.find("name").text for obj in objects]

        return object_names

    def create_data_structure(xml_files):
        grouped_data = {}

        for xml_path in xml_files:
            tree = ET.parse(xml_path)
            root_element = tree.getroot()
            size = (int(root_element.find("size/width").text), int(root_element.find("size/height").text))
            filename = os.path.basename(xml_path)

            if size not in grouped_data:
                grouped_data[size] = []

            object_names = parse_objects_in_file(xml_path)
            grouped_data[size].append({"Filename": filename, "Object Names": object_names})

        return grouped_data

    def format_object_names(object_names_list):
        formatted_names = []

        for names in object_names_list:
            formatted_names.append(", ".join(names))

        return formatted_names

    if __name__ == "__main__":
    
        xml_files = read_xml_files(xml_folder_path)
        grouped_data = create_data_structure(xml_files)

        data = []

        for size, files in grouped_data.items():
            size_data = {
                "Size": size,
                "Data": files
            }

            data.append(size_data)

        data_list = data

        result_dict = {}

        for idx, data in enumerate(data_list, start=1):
            filename_list = [entry['Filename'] for entry in data['Data']]
            object_names_list = format_object_names([entry['Object Names'] for entry in data['Data']])

            result = {
                "Filename": filename_list,
                "Object Names": object_names_list
            }
    
            result_dict[f"result{idx}"] = result
        
    result_keys = list(result_dict.keys())

    iteration_count = 0 
    for result_key in result_keys:
        data_dit = result_dict[result_key]
        stop_condition = False  # 반복을 멈출 조건 초기값 설정

    # 데이터 전처리 수행
        train_data_dict, test_data_dict, object_name_list = preprocess_data(data_dit,ratio)

        while not stop_condition and iteration_count < 10:  # 최대 10번 반복
    # 1번 코드
            object_name_counts = defaultdict(int)
            for object_names in train_data_dict.values():
                for object_name in object_names:
                    object_name_counts[object_name] += 1

            object_name_sums = {name.replace(" ", "_") + "_total": count for name, count in object_name_counts.items()}

            for variable_name, count in object_name_sums.items():
                globals()[variable_name] = count

            total_count = sum(object_name_sums.values())

            object_name_counts_test = defaultdict(int)
            for object_names in test_data_dict.values():
                for object_name in object_names:
                    object_name_counts_test[object_name] += 1

            object_name_sums_test = {name.replace(" ", "_") + "_totalt": count for name, count in object_name_counts_test.items()}

            for variable_name, count in object_name_sums_test.items():
                globals()[variable_name] = count

            total_count_test = sum(object_name_sums_test.values())

    # 2번 코드
            percentage_results, max_percentage_objects = calculate_percentage(train_data_dict, object_name_list)

    # 2번 결과 출력
            print_results_with_threshold(percentage_results, max_percentage_objects, 85)

            max_count = 0
            trade_train = None

            for key, value in train_data_dict.items():
                count = sum(1 for object_name in value if object_name in max_percentage_objects)
        
                if count > max_count:
                    max_count = count
                    trade_train = key

            if trade_train is not None:
                print(f"The key with the most occurrences is: {trade_train}")
            else:
                print("No key found.")

    # 3번 코드
            percentage_results_test, max_percentage_objects_test = calculate_percentage_test(test_data_dict, object_name_list)

    # 3번 결과 출력
            print_results_with_threshold(percentage_results_test, max_percentage_objects_test, 15)

            max_count_test = 0
            trade_test = None

            for key, value in test_data_dict.items():
                count_test = sum(1 for object_name in value if object_name in max_percentage_objects_test)
        
                if count_test > max_count_test:
                    max_count_test = count_test
                    trade_test = key

            if trade_test is not None:
                print(f"The key with the most occurrences is: {trade_test}")
            else:
                print("No key found.")

    # 4번 코드
            if trade_train in train_data_dict:
                test_data_dict[trade_train] = train_data_dict[trade_train]
                del train_data_dict[trade_train]

# 이동한 파일의 키를 train_data_dict에서 제거
            if trade_train in train_data_dict:
                del train_data_dict[trade_train]

# 5번 코드
            if trade_test in test_data_dict:
                train_data_dict[trade_test] = test_data_dict[trade_test]
                del test_data_dict[trade_test]

# 이동한 파일의 키를 test_data_dict에서 제거
            if trade_test in test_data_dict:
                del test_data_dict[trade_test]

    # 6번 코드
            percentage_results, max_percentage_objects = calculate_percentage(train_data_dict, object_name_list)

            min_rt_key = min(percentage_results, key=percentage_results.get)
            max_rt_key = max(percentage_results, key=percentage_results.get)

            min_rt = percentage_results[min_rt_key]
            max_rt = percentage_results[max_rt_key]

            print("min_rt:", min_rt_key, "(", min_rt, ")")
            print("max_rt:", max_rt_key, "(", max_rt, ")")
        
            iteration_count += 1

    # 반복을 멈출 조건 설정
            if min_rt > minin and max_rt < maxax:
                stop_condition = True

    # 반복 조건을 만족하지 않을 경우 #1번으로 돌아가는 것이 이미 while 루프의 동작 방식입니다.

        test_data_dict_name = list(test_data_dict.keys())
        train_data_dict_name = list(train_data_dict.keys())
        
        # Create the target directory if it doesn't exist
        os.makedirs(train_target_dir, exist_ok=True)
        os.makedirs(test_target_dir, exist_ok=True)

        for root, _, files in os.walk(xml_folder_path):
            for filename in files:
                if filename in train_data_dict_name:
                    source_path = os.path.join(root, filename)
                    target_path = os.path.join(train_target_dir, filename)
            
            # Check if the source file exists before copying
                    if os.path.exists(source_path):
                        shutil.copy(source_path, target_path)
                        print(f"File {filename} copied successfully.")
                    else:
                        print(f"File {filename} not found in source directory.")

# Move XML files to the test target directory based on filenames
        for root, _, files in os.walk(xml_folder_path):
            for filename in files:
                if filename in test_data_dict_name:
                    source_path_test = os.path.join(root, filename)
                    target_path_test = os.path.join(test_target_dir, filename)
            
            # Check if the source file exists before copying
                    if os.path.exists(source_path_test):
                        shutil.copy(source_path_test, target_path_test)
                        print(f"File {filename} copied to test directory successfully.")
                    else:
                        print(f"File {filename} not found in source directory.")

        test_data_dict_name = [
            filename.replace(".xml", ".jpg") for filename in test_data_dict_name]
        train_data_dict_name = [
            filename.replace(".xml", ".jpg") for filename in train_data_dict_name]

# 원본과 대상 디렉토리 경로


        if not os.path.exists(target_directory):
            os.makedirs(target_directory)

# 하위 폴더를 검색하며 일치하는 이미지를 복사
        for root, dirs, files in os.walk(source_directory):
            for file in files:
                if file in train_data_dict_name:
                    source_file_path = os.path.join(root, file)
                    target_file_path = os.path.join(target_directory, file)
                    shutil.copy(source_file_path, target_file_path)

# 대상 디렉토리가 존재하지 않으면 생성
        if not os.path.exists(target_directory_test):
            os.makedirs(target_directory_test)

# 하위 폴더를 검색하며 일치하는 이미지를 복사
        for root, dirs, files in os.walk(source_directory):
            for file in files:
                if file in test_data_dict_name:
                    source_file_path = os.path.join(root, file)
                    target_file_path = os.path.join(target_directory_test, file)
            
                    if os.path.exists(source_file_path):  # 일치하는 이미지가 있는지 확인
                        shutil.copy(source_file_path, target_file_path)
                    else:
                        print(f"이미지를 찾을 수 없어 {file}을(를) 건너뜁니다.")

# 결과 출력
        print(f"{result_key}의 반복이 완료되었습니다.")

    # 각 반복 이후, result_dict 항목을 모두 처리했는지 확인합니다.
        if result_keys.index(result_key) < len(result_keys) - 1:
            next_result_key = result_keys[result_keys.index(result_key) + 1]
            data_dit = result_dict[next_result_key]
        else:
        # 모든 반복이 완료된 경우
            print("All iterations completed.")
            break  # 모든 반복이 완료되었으면 루프를 종료합니다.

    # 반복 횟수 증가
        iteration_count += 1
    


In [4]:
minin = 77   #오차 최소값
maxax = 83   #오차 최대값
ratio = 0.8   #train, test 비율

xml_folder_path = "C:/work/python/fp/archive/PCB_DATASET/Annotations"           #xml 경로
source_directory = r'C:/work/python/fp/archive/PCB_DATASET/images'              #이미지 경로
target_directory = r'C:/work/python/fp/archive/PCB_DATASET/train/images'           #train 이미지 생성 경로
target_directory_test = r'C:/work/python/fp/archive/PCB_DATASET/test/images'       #test 이미지 생성 경로
train_target_dir = "C:/work/python/fp/archive/PCB_DATASET/train/xml"               #xml train 경로
test_target_dir = "C:/work/python/fp/archive/PCB_DATASET/test/xml"                 #xml test 경로



trade_test_model()
image_traintest()


Object with the highest percentage above 85%: spur  96.23%
The key with the most occurrences is: 01_spur_07.xml

Object with the highest percentage above 15%: missing_hole  31.15%
The key with the most occurrences is: 01_missing_hole_05.xml
min_rt: missing_hole ( 68.85245901639344 )
max_rt: spur ( 96.22641509433963 )

Object with the highest percentage above 85%: spur  90.57%
The key with the most occurrences is: 01_spur_10.xml

Object with the highest percentage above 15%: open_circuit  25.42%
The key with the most occurrences is: 01_open_circuit_19.xml
min_rt: open_circuit ( 74.57627118644068 )
max_rt: spur ( 90.56603773584906 )
No objects with percentage above 85% found.
The key with the most occurrences is: 01_spur_11.xml

Object with the highest percentage above 15%: missing_hole  24.59%
The key with the most occurrences is: 01_missing_hole_19.xml
min_rt: missing_hole ( 75.40983606557377 )
max_rt: spur ( 84.90566037735849 )
No objects with percentage above 85% found.
The key with

result2의 반복이 완료되었습니다.
File 05_missing_hole_01.xml copied successfully.
File 05_missing_hole_03.xml copied successfully.
File 05_missing_hole_05.xml copied successfully.
File 05_missing_hole_06.xml copied successfully.
File 05_missing_hole_07.xml copied successfully.
File 05_missing_hole_08.xml copied successfully.
File 05_mouse_bite_01.xml copied successfully.
File 05_mouse_bite_02.xml copied successfully.
File 05_mouse_bite_03.xml copied successfully.
File 05_mouse_bite_04.xml copied successfully.
File 05_mouse_bite_07.xml copied successfully.
File 05_mouse_bite_08.xml copied successfully.
File 05_mouse_bite_09.xml copied successfully.
File 05_mouse_bite_10.xml copied successfully.
File 05_open_circuit_01.xml copied successfully.
File 05_open_circuit_03.xml copied successfully.
File 05_open_circuit_04.xml copied successfully.
File 05_open_circuit_05.xml copied successfully.
File 05_open_circuit_06.xml copied successfully.
File 05_open_circuit_07.xml copied successfully.
File 05_open_c

result5의 반복이 완료되었습니다.
File 08_missing_hole_01.xml copied successfully.
File 08_missing_hole_02.xml copied successfully.
File 08_missing_hole_03.xml copied successfully.
File 08_missing_hole_04.xml copied successfully.
File 08_missing_hole_07.xml copied successfully.
File 08_missing_hole_09.xml copied successfully.
File 08_missing_hole_10.xml copied successfully.
File 08_mouse_bite_01.xml copied successfully.
File 08_mouse_bite_02.xml copied successfully.
File 08_mouse_bite_03.xml copied successfully.
File 08_mouse_bite_04.xml copied successfully.
File 08_mouse_bite_05.xml copied successfully.
File 08_mouse_bite_06.xml copied successfully.
File 08_mouse_bite_07.xml copied successfully.
File 08_mouse_bite_08.xml copied successfully.
File 08_mouse_bite_09.xml copied successfully.
File 08_mouse_bite_10.xml copied successfully.
File 08_open_circuit_02.xml copied successfully.
File 08_open_circuit_04.xml copied successfully.
File 08_open_circuit_05.xml copied successfully.
File 08_open_circu

File 11_spurious_copper_05.xml copied to test directory successfully.
File 11_spurious_copper_07.xml copied to test directory successfully.
result9의 반복이 완료되었습니다.
File 12_missing_hole_01.xml copied successfully.
File 12_missing_hole_03.xml copied successfully.
File 12_missing_hole_04.xml copied successfully.
File 12_missing_hole_05.xml copied successfully.
File 12_missing_hole_06.xml copied successfully.
File 12_missing_hole_07.xml copied successfully.
File 12_missing_hole_08.xml copied successfully.
File 12_missing_hole_10.xml copied successfully.
File 12_mouse_bite_02.xml copied successfully.
File 12_mouse_bite_03.xml copied successfully.
File 12_mouse_bite_04.xml copied successfully.
File 12_mouse_bite_05.xml copied successfully.
File 12_mouse_bite_07.xml copied successfully.
File 12_mouse_bite_08.xml copied successfully.
File 12_mouse_bite_09.xml copied successfully.
File 12_mouse_bite_10.xml copied successfully.
File 12_open_circuit_01.xml copied successfully.
File 12_open_circuit_